In [9]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

operations = [

    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),

    UpdateOne({'course': 'CS 102'}, {'$set': {'enrollments': 28}}),
    UpdateOne({'course': 'Math 101'}, {'$inc': {'enrollments': 5}}),

    DeleteOne({'course': 'History 201'})
]

result = courses_collection.bulk_write(operations)


print("BULK OPERATION SUMMARY")
print("Inserted:", result.inserted_count)
print("Modified:", result.modified_count)
print("Deleted:", result.deleted_count)


print("\nCURRENT COURSES DATA")
for doc in courses_collection.find():
    print(doc)


BULK OPERATION SUMMARY
Inserted: 4
Modified: 2
Deleted: 1

CURRENT COURSES DATA
{'_id': ObjectId('6912e486be0c18ade907f672'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('6912e486be0c18ade907f673'), 'course': 'CS 102', 'enrollments': 28, 'department': 'Computer Science'}
{'_id': ObjectId('6912e486be0c18ade907f675'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}


In [18]:
print("Query for courses with enrollments greater than 20")
for course in courses_collection.find({'enrollments': {'$gt': 28}}):
    print(course)

print ("Query courses in Computer Science or Mathematics departments")
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

print("nilai 28 termasuk dalam himpunan")
for course in courses_collection.find({'enrollments': {'$gte': 28}}):
    print(course)


Query for courses with enrollments greater than 20
{'_id': ObjectId('6912e486be0c18ade907f672'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
Query courses in Computer Science or Mathematics departments
{'_id': ObjectId('6912e486be0c18ade907f672'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('6912e486be0c18ade907f673'), 'course': 'CS 102', 'enrollments': 28, 'department': 'Computer Science'}
nilai 28 termasuk dalam himpunan
{'_id': ObjectId('6912e486be0c18ade907f672'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('6912e486be0c18ade907f673'), 'course': 'CS 102', 'enrollments': 28, 'department': 'Computer Science'}


In [23]:
print ("verage enrollment per department using aggregation")
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


max_pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}},
    {'$sort': {'_id': 1}}
]

print("\n=== Maximum Enrollment per Department ===")
for result in courses_collection.aggregate(max_pipeline):
    print(f"Department: {result['_id']} - Max Enrollment: {result['max_enrollment']}")


verage enrollment per department using aggregation
{'_id': 'Computer Science', 'average_enrollment': 28.0}
{'_id': 'Mathematics', 'average_enrollment': 35.0}
{'_id': 'Physics', 'average_enrollment': 15.0}

=== Maximum Enrollment per Department ===
Department: Computer Science - Max Enrollment: 28
Department: Mathematics - Max Enrollment: 35
Department: Physics - Max Enrollment: 15


In [28]:
print("Projection to rename fields")
pipeline = [
    {'$project': {'_id': 0, 'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

print("\nAdding enrollment category field based on enrollments")
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


Projection to rename fields
{'enrollments': 35, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'enrollments': 28, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}

Adding enrollment category field based on enrollments
{'_id': ObjectId('6912e486be0c18ade907f672'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('6912e486be0c18ade907f673'), 'course': 'CS 102', 'enrollments': 28, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('6912e486be0c18ade907f675'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


In [29]:
from pymongo import InsertOne

bulk_courses = [
    # === Computer Science ===
    InsertOne({'course': 'CS 101', 'enrollments': 28, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 102', 'enrollments': 32, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 103', 'enrollments': 21, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 104', 'enrollments': 19, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 105', 'enrollments': 40, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 106', 'enrollments': 27, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 107', 'enrollments': 35, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 108', 'enrollments': 22, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 109', 'enrollments': 31, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 110', 'enrollments': 24, 'department': 'Computer Science'}),

    # === Mathematics ===
    InsertOne({'course': 'Math 201', 'enrollments': 17, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 202', 'enrollments': 26, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 203', 'enrollments': 34, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 204', 'enrollments': 20, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 205', 'enrollments': 29, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 206', 'enrollments': 23, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 207', 'enrollments': 15, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 208', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 209', 'enrollments': 33, 'department': 'Mathematics'}),
    InsertOne({'course': 'Math 210', 'enrollments': 25, 'department': 'Mathematics'}),

    # === Physics ===
    InsertOne({'course': 'Physics 301', 'enrollments': 14, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 302', 'enrollments': 18, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 303', 'enrollments': 22, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 304', 'enrollments': 37, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 305', 'enrollments': 19, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 306', 'enrollments': 24, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 307', 'enrollments': 33, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 308', 'enrollments': 28, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 309', 'enrollments': 16, 'department': 'Physics'}),
    InsertOne({'course': 'Physics 310', 'enrollments': 40, 'department': 'Physics'}),

    # === Biology ===
    InsertOne({'course': 'Biology 401', 'enrollments': 12, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 402', 'enrollments': 17, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 403', 'enrollments': 29, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 404', 'enrollments': 25, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 405', 'enrollments': 31, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 406', 'enrollments': 15, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 407', 'enrollments': 38, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 408', 'enrollments': 27, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 409', 'enrollments': 20, 'department': 'Biology'}),
    InsertOne({'course': 'Biology 410', 'enrollments': 34, 'department': 'Biology'}),

    # === History ===
    InsertOne({'course': 'History 501', 'enrollments': 13, 'department': 'History'}),
    InsertOne({'course': 'History 502', 'enrollments': 19, 'department': 'History'}),
    InsertOne({'course': 'History 503', 'enrollments': 23, 'department': 'History'}),
    InsertOne({'course': 'History 504', 'enrollments': 30, 'department': 'History'}),
    InsertOne({'course': 'History 505', 'enrollments': 27, 'department': 'History'}),
    InsertOne({'course': 'History 506', 'enrollments': 22, 'department': 'History'}),
    InsertOne({'course': 'History 507', 'enrollments': 18, 'department': 'History'}),
    InsertOne({'course': 'History 508', 'enrollments': 25, 'department': 'History'}),
    InsertOne({'course': 'History 509', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'History 510', 'enrollments': 35, 'department': 'History'}),
]

courses_collection.bulk_write(bulk_courses)
print(" 50 manual course documents inserted successfully.")


 50 manual course documents inserted successfully.


In [30]:
print("\nHomework 1: Count of courses per department")
pipeline = [
    {'$group': {'_id': '$department', 'course_count': {'$sum': 1}}},
    {'$sort': {'course_count': -1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


Homework 1: Count of courses per department
{'_id': 'Physics', 'course_count': 11}
{'_id': 'Mathematics', 'course_count': 11}
{'_id': 'Computer Science', 'course_count': 11}
{'_id': 'History', 'course_count': 10}
{'_id': 'Biology', 'course_count': 10}


In [31]:
print("\nHomework 2: Courses >25 enrollments in Computer Science")
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$group': {'_id': '$department', 'total_courses': {'$sum': 1}, 'avg_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


Homework 2: Courses >25 enrollments in Computer Science
{'_id': 'Computer Science', 'total_courses': 7, 'avg_enrollment': 31.571428571428573}


In [33]:
students_collection = db['students']
bulk_students = [
    InsertOne({'student_name': 'Alice', 'course': 'CS 101'}),
    InsertOne({'student_name': 'Bob', 'course': 'CS 102'}),
    InsertOne({'student_name': 'Charlie', 'course': 'Math 202'}),
    InsertOne({'student_name': 'David', 'course': 'Physics 304'}),
    InsertOne({'student_name': 'Eve', 'course': 'Biology 405'}),
    InsertOne({'student_name': 'Frank', 'course': 'History 504'}),
    InsertOne({'student_name': 'Grace', 'course': 'CS 107'}),
    InsertOne({'student_name': 'Helen', 'course': 'Math 208'}),
    InsertOne({'student_name': 'Ian', 'course': 'Physics 307'}),
    InsertOne({'student_name': 'Julia', 'course': 'History 510'}),
]

students_collection.bulk_write(bulk_students)
print(" 10 manual student documents inserted successfully.")

print("\nHomework 3: Join courses with students using $lookup")
pipeline = [
    {
        '$lookup': {
            'from': 'students',
            'localField': 'course',
            'foreignField': 'course',
            'as': 'student_details'
        }
    },
    {'$project': {'_id': 0, 'course': 1, 'department': 1, 'student_details.student_name': 1}},
    {'$limit': 10}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


 10 manual student documents inserted successfully.

Homework 3: Join courses with students using $lookup
{'course': 'Math 101', 'department': 'Mathematics', 'student_details': []}
{'course': 'CS 102', 'department': 'Computer Science', 'student_details': [{'student_name': 'Bob'}, {'student_name': 'Bob'}]}
{'course': 'Physics 202', 'department': 'Physics', 'student_details': []}
{'course': 'CS 101', 'department': 'Computer Science', 'student_details': [{'student_name': 'Alice'}, {'student_name': 'Alice'}]}
{'course': 'CS 102', 'department': 'Computer Science', 'student_details': [{'student_name': 'Bob'}, {'student_name': 'Bob'}]}
{'course': 'CS 103', 'department': 'Computer Science', 'student_details': []}
{'course': 'CS 104', 'department': 'Computer Science', 'student_details': []}
{'course': 'CS 105', 'department': 'Computer Science', 'student_details': []}
{'course': 'CS 106', 'department': 'Computer Science', 'student_details': []}
{'course': 'CS 107', 'department': 'Computer Scienc